<a href="https://colab.research.google.com/github/HasankaRajakaruna/Kaggle-Projects/blob/main/Model_One.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [95]:
import pandas as pd
from datetime import datetime
import numpy as np

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [100]:
train = pd.read_csv('/content/drive/MyDrive/Dataset/dataset_gbpusd_train.csv')
test = pd.read_csv('/content/drive/MyDrive/Dataset/dataset_gbpusd_test.csv')

In [101]:
train.shape

(21715, 10)

In [102]:
train.head()

,time,Unnamed: 0,open,high,low,close,tick_volume,spread,real_volume,target
0,2010-01-04 00:00:00,0,1.61210,1.61486,1.60852,1.61087,7471,24,0,2
1,2010-01-04 04:00:00,1,1.61087,1.61090,1.60691,1.60850,5966,24,0,2
2,2010-01-04 08:00:00,2,1.60851,1.62380,1.60554,1.61744,13700,24,0,2
3,2010-01-04 12:00:00,3,1.61748,1.62245,1.61375,1.61656,12921,24,0,2
4,2010-01-04 16:00:00,4,1.61651,1.61877,1.60822,1.60958,10789,23,0,2


In [103]:
test.shape

(384, 11)

In [104]:
test.head()

,Unnamed: 0.1,Unnamed: 0,time,open,high,low,close,tick_volume,spread,real_volume,target
0,0,0,2024-01-02 00:00:00,1.27205,1.27355,1.27173,1.27210,5319,0,0,2
1,1,1,2024-01-02 04:00:00,1.27210,1.27414,1.27100,1.27370,6880,0,0,2
2,2,2,2024-01-02 08:00:00,1.27370,1.27602,1.27174,1.27176,13263,0,0,2
3,3,3,2024-01-02 12:00:00,1.27176,1.27188,1.26217,1.26300,16548,0,0,2
4,4,4,2024-01-02 16:00:00,1.26291,1.26448,1.26108,1.26330,18454,0,0,2


In [112]:
dataset = pd.concat([train,test])

In [106]:
dataset.shape

(22099, 11)

In [107]:
duplicates = dataset[dataset.duplicated()]

print("Duplicate Rows:")
print(duplicates)

Duplicate Rows:
Empty DataFrame
Columns: [time, Unnamed: 0, open, high, low, close, tick_volume, spread, real_volume, target, Unnamed: 0.1]
Index: []


Calculate Target Value

In [113]:
array_with_zeros = np.zeros(len(dataset), dtype=int)

def calculate_target(data):

    for a in range(len(data)):
        max_value = data['high'].iloc[a]
        buy_price = data['open'].iloc[a]

        for b in range(a, a + 3, 1):
            if b < len(data):
                max_value = max(max_value, data['high'].iloc[b])
                diff_buy = (max_value - buy_price) * 10000

                if diff_buy >= 30:
                    array_with_zeros[a] = 1
                    break

calculate_target(dataset)


Target column

In [115]:
dataset.reset_index(drop=True, inplace=True)  # Reset the index of the DataFrame

target_column = pd.DataFrame({'target': array_with_zeros})

if 'target' in dataset.columns:
    dataset.drop(columns=['target'], inplace=True)

dataset = pd.concat([dataset, target_column], axis=1)

time            2010-01-04 00:00:00
Unnamed: 0                        0
open                         1.6121
high                        1.61486
low                         1.60852
close                       1.61087
tick_volume                    7471
spread                           24
real_volume                       0
target                            2
Unnamed: 0.1                    NaN
Name: 0, dtype: object


In [116]:
dataset.head()

,time,Unnamed: 0,open,high,low,close,tick_volume,spread,real_volume,Unnamed: 0.1,target
0,2010-01-04 00:00:00,0,1.61210,1.61486,1.60852,1.61087,7471,24,0,NaN,1
1,2010-01-04 04:00:00,1,1.61087,1.61090,1.60691,1.60850,5966,24,0,NaN,1
2,2010-01-04 08:00:00,2,1.60851,1.62380,1.60554,1.61744,13700,24,0,NaN,1
3,2010-01-04 12:00:00,3,1.61748,1.62245,1.61375,1.61656,12921,24,0,NaN,1
4,2010-01-04 16:00:00,4,1.61651,1.61877,1.60822,1.60958,10789,23,0,NaN,0


Heikinashi candlestics

In [ ]:
def calculate_heikin_ashi(data):
    data["HA_Close"] = (data["open"] + data["high"] + data["low"] + data["close"]) / 4
    data["HA_Open"] = (data["open"].shift(1) + data["close"].shift(1)) / 2
    data["HA_High"] = data[["high", "close", "open"]].max(axis=1)
    data["HA_Low"] = data[["low", "close", "open"]].min(axis=1)

calculate_heikin_ashi(dataset)

In [ ]:
dataset = dataset[['HA_Open', 'HA_High', 'HA_Low','HA_Close','target']]

In [ ]:
dataset.head()

In [ ]:
close_prices = dataset['HA_Close'].values
high_prices = dataset['HA_High'].values
low_prices = dataset['HA_Low'].values
open_prices = dataset['HA_Open'].values

Indicators

In [ ]:
import talib as ta

In [ ]:
!wget http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz
!tar -xzvf ta-lib-0.4.0-src.tar.gz
%cd ta-lib
!./configure --prefix=/usr
!make
!make install
!pip install Ta-Lib

In [ ]:
#Ema indicator
ema_period5 = 5
dataset['EMA5'] = ta.EMA(close_prices, timeperiod=ema_period5)

ema_period10 = 10
dataset['EMA10'] = ta.EMA(close_prices, timeperiod=ema_period10)

ema_period20 = 20
dataset['EMA20'] = ta.EMA(close_prices, timeperiod=ema_period20)

ema_period50 = 50
dataset['EMA50'] = ta.EMA(close_prices, timeperiod=ema_period50)

ema_period100 = 100
dataset['EMA100'] = ta.EMA(close_prices, timeperiod=ema_period100)

ema_period200 = 200
dataset['EMA200'] = ta.EMA(close_prices, timeperiod=ema_period200)

Duplicate Data

In [ ]:
duplicates = dataset[dataset.duplicated()]

print("Duplicate Rows:")
print(duplicates)

In [ ]:
dataset.drop_duplicates(inplace=True)

Missing Values

In [ ]:
missing_values = dataset.isnull().sum()

print("Columns with Missing Values:")
print(missing_values[missing_values > 0])

In [ ]:
dataset.dropna(inplace=True)

In [ ]:
dataset

Model Traning

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import joblib
from sklearn.metrics import accuracy_score

Train set and Test set

In [ ]:
test_num_rows = test.shape[0]
test_data = dataset.tail(test_num_rows)
train_num_rows = dataset.shape[0]
train_data = dataset.head(train_num_rows-test_num_rows)

In [ ]:
value_counts = train_data['target'].value_counts()
value_counts

In [ ]:
#buy orders
bb_order = train_data[train_data['target'] == 1].tail(5000)
bb_order

In [ ]:
#sell orders
ss_order = train_data[train_data['target'] == 0].tail(5000)

In [ ]:
new_dataset = pd.concat([bb_order, ss_order], ignore_index=True)
new_dataset

In [ ]:
X_data=new_dataset.drop(['target'],axis=1)
y_data=new_dataset[['target']]

Scaling Data

In [ ]:
scale = MinMaxScaler()
data_set_scaled = scale.fit_transform(X_data)
print(data_set_scaled)

Model

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.ensemble import RandomForestClassifier

X_train, X_test, y_train, y_test = train_test_split(data_set_scaled, y_data, test_size=0.2, random_state=42)
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
cv_scores = cross_val_score(rf_classifier, X_train, y_train, cv=5)  # 5-fold cross-validation
rf_classifier.fit(X_train, y_train)
test_score = rf_classifier.score(X_test, y_test)


print("Cross-Validation Scores:", cv_scores)
print("Mean CV Score:", np.mean(cv_scores))
print("Test Set Score:", test_score)